In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import itertools
import sklearn
import pandas as pd

In [2]:
neuron_data = {}



In [3]:
files = os.listdir('../neuron_logs/train_data')

In [4]:
with open(os.path.join('..', 'neuron_logs', 'train_data', 'output_20190923-192628.json'), 'r') as f:
    neuron_data = json.load(f)

target = 'usefulness_loss'


target_thr = [neuron_data[e][neuron][target] for neuron in neuron_data['0'] for e in neuron_data  if e != '0' and ' ' in neuron]

threshold = np.percentile(target_thr, 80) # median

print(threshold)



1.4357221603393555


In [5]:
print(neuron_data['1']['0 0'].keys())
np.array(neuron_data['1']['3 0']['output_weights']).shape

dict_keys(['activations', 'usefulness_loss', 'usefulness_acc', 'depth', 'inverse_depth', 'width', 'input_weights', 'output_weights', 'reg_loss_in_layer'])


(10,)

In [6]:
train, test = [], []
for i, filename in enumerate(files):
    with open(os.path.join('..', 'neuron_logs', 'train_data', filename), 'r') as f:
        neuron_data = json.load(f)
    if i == 10:
        break
    for e in neuron_data.keys():
        if e == '0':
            continue
        for neuron in neuron_data[e]:
            if ' ' not in neuron:
                continue
            current_data = neuron_data[e][neuron]
            important_features = []
            important_features += current_data['activations'] #HACKY
            important_features += [current_data['depth']]
            important_features += [current_data['inverse_depth']]
            important_features += [current_data['width']]
            # important_features += [current_data['input_weights']]
            # important_features += [current_data['output_weights']]
            important_features += [current_data['reg_loss_in_layer']]
            train += [important_features]
            test += [int(current_data[target] > threshold)]

In [7]:
train = np.array(train)
test = np.array(test)

In [8]:
test.shape

(12000,)

In [9]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(train, test, test_size=0.2, random_state=0)
print(f"{X_train.shape} {X_test.shape} {y_train.shape} {y_test.shape}")
clf = LogisticRegression().fit(X_train, y_train)
clf.score(X_test, y_test) 

(9600, 10004) (2400, 10004) (9600,) (2400,)


/home/levaid/.miniconda3/envs/tensorflow_env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9166666666666666

In [10]:
clf.coef_

array([[-0.43740408,  0.22133338, -0.15104726, ...,  0.55883037,
        -0.02457438, -0.87689349]])

In [23]:
clf.score(X_train, y_train)

0.9746875

In [22]:
np.argsort(np.abs(clf.coef_))
clf.coef_[0][10000]

-2.8395456822869694

In [21]:
sorted(np.abs(clf.coef_)[0], reverse=True)

[2.8395456822869694,
 1.3790126471531663,
 1.233026938200898,
 1.1788964159023552,
 1.169778870595428,
 1.1107628495191193,
 1.0925813581441761,
 1.0369119099911814,
 1.0358190721629266,
 1.0308028540384997,
 1.0278435317607568,
 1.0254025558796416,
 1.0144973278890888,
 1.0086137510673947,
 1.0073272608649575,
 0.981030935481365,
 0.9746031097921033,
 0.9681847511808331,
 0.9533788076660197,
 0.9368682250959717,
 0.9356862163976339,
 0.9261430997231216,
 0.9221374669503697,
 0.9215201593522442,
 0.9164828610551535,
 0.900742245084962,
 0.890482457202329,
 0.8889294080039101,
 0.8809163522028765,
 0.8787681160332514,
 0.8768934856231853,
 0.8756760256297057,
 0.8722922449725693,
 0.8658092668094484,
 0.863539594774268,
 0.8578188728053048,
 0.8574098157111646,
 0.8565920354540937,
 0.8545695529618069,
 0.8531219699658614,
 0.8488268597962269,
 0.8474981656071005,
 0.8458276945587876,
 0.8407834143850339,
 0.8364853527869328,
 0.8323084498553536,
 0.8311638939817807,
 0.8296804411553336

In [16]:
clf.coef_[0]

array([-0.43740408,  0.22133338, -0.15104726, ...,  0.55883037,
       -0.02457438, -0.87689349])